# Extraction des lettres

In [11]:
# Import
from os import listdir
import datetime as dt
import pandas as pd
import numpy as np
import json
from io import StringIO, BytesIO

In [17]:
# Constitution du dataframe questions_df
data = []
dossier_questions = 'questionsXV'
for i, file in enumerate(listdir(dossier_questions)):
    try:
        with open(dossier_questions+'/'+file, 'r') as f:
            # Code de Xavier Tannier pour ouvrir les fichiers avec le bon encodage
            content = f.read()
            io = StringIO(content)
            json_data = json.load(io)

            # print(json.dumps(json_data, indent=4, sort_keys=True))

            question = json_data["question"]["textesQuestion"]["texteQuestion"]["texte"]
            groupe_auteur = json_data["question"]["auteur"]["groupe"]["abrege"]
            rubrique = json_data["question"]["indexationAN"]["rubrique"]
            titre = json_data["question"]["indexationAN"]["ANALYSE"]["ANA"]
            #texte = json_data["question"]["textesQuestion"]["texteQuestion"]
            try:
                date_q = json_data["question"]["textesQuestion"]["texteQuestion"]["infoJO"]["dateJO"]
                date_question = dt.datetime.strptime(date_q, "%d/%m/%Y")
            except:
                print('pb q'  + file)
            if json_data["question"].get("textesReponse"):
                # Certaines questions n'ont pas encore de réponse... 
                try:
                    date_r = json_data["question"]["textesReponse"]["texteReponse"]["infoJO"]["dateJO"]
                    if len(date_r) > 0:
                        date_reponse = dt.datetime.strptime(date_r, "%d/%m/%Y")
                    else:
                         date_reponse = None
                except:
                    pass
                    #print(file) 

            else:
                date_reponse = None
            ministere_adresse = json_data["question"]["minInt"]["developpe"]

            # On ajoute les colonnes qui nous intéressent dans une liste de dictionnaire : data
            data.append({
                "question": question,
                "groupe_auteur": groupe_auteur,
                "date_question": date_question,
                "date_reponse": date_reponse,
                "ministere_adresse": ministere_adresse,
                "rubrique": rubrique,
                "titre":titre,
                #"texte":texte,
            })
    except:
        pass
        
questions_df = pd.DataFrame(data)
questions_df.head(10)

,date_question,date_reponse,groupe_auteur,ministere_adresse,question,rubrique,titre
0,2019-04-23,NaT,LAREM,Ministère de la justice,M. Jean-Charles Colas-Roy attire l'attention d...,droits fondamentaux,Reconnaissance de la langue des signes françai...
1,2018-06-12,2018-12-25,LAREM,Ministère de l’intérieur,M. Bertrand Sorre attire l'attention de M. le ...,sécurité routière,Précision sur les 80 km/h
2,2018-06-19,2018-10-02,LAREM,Ministère de la transition écologique et solid...,Mme Françoise Dumas attire l'attention de M. l...,eau et assainissement,Aides des agences de l'eau aux projets de réha...
3,2019-07-02,NaT,FI,Ministère de l'intérieur,M. Alexis Corbière alerte M. le ministre de l'...,sécurité des biens et des personnes,Conditions de travail des pompiers professionn...
4,2018-05-01,2018-09-25,LAREM,Ministère de l’intérieur,Mme Barbara Pompili appelle l'attention de M. ...,sécurité routière,Forfait post-stationnement et loueurs courte d...
5,2019-04-09,NaT,GDR,Ministère de l'action et des comptes publics,M. André Chassaigne attire l'attention de M. l...,professions de santé,Statut des conducteurs ambulanciers hospitalie...
6,2018-10-02,2019-05-07,SOC,Ministère de la justice,M. Régis Juanico interroge Mme la garde des sc...,ministères et secrétariats d'État,Rémunérations des membres du cabinet de la min...
7,2017-12-12,2018-04-10,FI,Ministère de l'éducation nationale,Mme Sabine Rubin attire l'attention de M. le m...,enseignement secondaire,Absence de collège public et laïque sur la com...
8,2019-03-05,NaT,LAREM,"Secrétariat d'État auprès du Premier ministre,...",M. Gaël Le Bohec interroge Mme la secrétaire d...,assurance maladie maternité,Grossesse - Prise en charge par la sécurité so...
9,2018-10-16,NaT,LR,Ministère des solidarités et de la santé,M. Gérard Menuel attire l'attention de Mme la ...,fonction publique hospitalière,Facturation du SMUR par les établissements hos...


In [23]:
# Les dates aux bons formats
questions_df['date_question'] = pd.to_datetime(questions_df['date_question'])
questions_df['date_reponse'] = pd.to_datetime(questions_df['date_reponse'])

In [24]:
# Save data
questions_df.to_pickle('samy.pkl')